In [1]:
%pip install torch
%pip install transformers
%pip install numpy
%pip install huggingface_hub
%pip install 'accelerate>=0.26.0'
%pip install --upgrade transformers

from huggingface_hub import login
import local_config

login(token=local_config.TOKEN)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


/anaconda/envs/azureml_py38_PT_TF/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/azureuser/.cache/huggingface/token
Login successful


In [2]:
#import torch
#import torch.nn as nn
#import torch.nn.functional as F
#from transformers import MllamaForConditionalGeneration

#original_model = MllamaForConditionalGeneration.from_pretrained("meta-llama/Llama-3.2-11B-Vision")


In [3]:

    #print("original_model", original_model)

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import MllamaForConditionalGeneration
import math
from collections import OrderedDict
from transformers import AutoConfig, AutoModelForCausalLM
from dataclasses import dataclass

pretrained_model = MllamaForConditionalGeneration.from_pretrained("meta-llama/Llama-3.2-11B-Vision")
pretrained_config = pretrained_model.config

print("pretrained_model", pretrained_model)

Loading checkpoint shards: 100%|██████████| 5/5 [00:41<00:00,  8.39s/it]

pretrained_model MllamaForConditionalGeneration(
  (vision_model): MllamaVisionModel(
    (patch_embedding): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), padding=valid, bias=False)
    (gated_positional_embedding): MllamaPrecomputedPositionEmbedding(
      (tile_embedding): Embedding(9, 5248000)
    )
    (pre_tile_positional_embedding): MllamaPrecomputedAspectRatioEmbedding(
      (embedding): Embedding(9, 5120)
    )
    (post_tile_positional_embedding): MllamaPrecomputedAspectRatioEmbedding(
      (embedding): Embedding(9, 5120)
    )
    (layernorm_pre): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    (layernorm_post): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    (transformer): MllamaVisionEncoder(
      (layers): ModuleList(
        (0-31): 32 x MllamaVisionEncoderLayer(
          (self_attn): MllamaVisionSdpaAttention(
            (q_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (k_proj): Linear(in_features=1280, out

In [7]:
class MllamaForConditionalGeneration(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.vision_model_1 = MllamaVisionModel(config)
        self.vision_model_2 = MllamaVisionModel(config)
        self.vision_cross_attention = MllamaVisionCrossAttention(config)
        self.language_model = MllamaForCausalLM(config)
        self.multi_modal_projector = nn.Linear(config.vision_hidden_size * 2, config.hidden_size)

    def forward(self, image_1, image_2, input_ids, attention_mask=None):
        vision_output_1 = self.vision_model_1(image_1)
        vision_output_2 = self.vision_model_2(image_2)
        
        # Cross-attention between vision models
        vision_output_1 = self.vision_cross_attention(vision_output_1, vision_output_2)
        vision_output_2 = self.vision_cross_attention(vision_output_2, vision_output_1)
        
        # Concatenate outputs from both vision models
        combined_vision_output = torch.cat([vision_output_1, vision_output_2], dim=-1)
        
        # Project combined vision output to language model dimension
        projected_vision_output = self.multi_modal_projector(combined_vision_output)
        
        # Combine with text input and pass through language model
        lm_output = self.language_model(input_ids, attention_mask, projected_vision_output)
        
        return lm_output

class MllamaVisionCrossAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.query = nn.Linear(config.vision_hidden_size, config.vision_hidden_size)
        self.key = nn.Linear(config.vision_hidden_size, config.vision_hidden_size)
        self.value = nn.Linear(config.vision_hidden_size, config.vision_hidden_size)
        self.num_attention_heads = config.num_vision_attention_heads
        self.attention_head_size = config.vision_hidden_size // config.num_vision_attention_heads
        self.all_head_size = self.num_attention_heads * self.attention_head_size
        
    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)
        
    def forward(self, hidden_states, context):
        mixed_query_layer = self.query(hidden_states)
        mixed_key_layer = self.key(context)
        mixed_value_layer = self.value(context)

        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(mixed_key_layer)
        value_layer = self.transpose_for_scores(mixed_value_layer)

        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        attention_probs = nn.Softmax(dim=-1)(attention_scores)

        context_layer = torch.matmul(attention_probs, value_layer)
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)

        return context_layer

class MllamaVisionModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.patch_embedding = nn.Conv2d(3, config.vision_hidden_size, kernel_size=config.patch_size, stride=config.patch_size)
        self.positional_embedding = nn.Parameter(torch.zeros(1, (config.image_size // config.patch_size) ** 2, config.vision_hidden_size))
        self.transformer = nn.ModuleList([MllamaVisionEncoderLayer(config) for _ in range(config.num_vision_layers)])
        self.layernorm = nn.LayerNorm(config.vision_hidden_size)

    def forward(self, pixel_values):
        batch_size = pixel_values.shape[0]
        x = self.patch_embedding(pixel_values).flatten(2).transpose(1, 2)
        x = x + self.positional_embedding
        for layer in self.transformer:
            x = layer(x)
        x = self.layernorm(x)
        return x

class MllamaVisionEncoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.attention = MllamaVisionSdpaAttention(config)
        self.mlp = MllamaVisionMLP(config)
        self.layernorm1 = nn.LayerNorm(config.vision_hidden_size)
        self.layernorm2 = nn.LayerNorm(config.vision_hidden_size)

    def forward(self, hidden_states):
        attention_output = self.attention(self.layernorm1(hidden_states))
        hidden_states = hidden_states + attention_output
        mlp_output = self.mlp(self.layernorm2(hidden_states))
        hidden_states = hidden_states + mlp_output
        return hidden_states

class MllamaVisionSdpaAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.num_attention_heads = config.num_vision_attention_heads
        self.attention_head_size = config.vision_hidden_size // config.num_vision_attention_heads
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = nn.Linear(config.vision_hidden_size, self.all_head_size)
        self.key = nn.Linear(config.vision_hidden_size, self.all_head_size)
        self.value = nn.Linear(config.vision_hidden_size, self.all_head_size)

        self.out_proj = nn.Linear(config.vision_hidden_size, config.vision_hidden_size)

    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(self, hidden_states):
        query_layer = self.transpose_for_scores(self.query(hidden_states))
        key_layer = self.transpose_for_scores(self.key(hidden_states))
        value_layer = self.transpose_for_scores(self.value(hidden_states))

        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        attention_probs = nn.Softmax(dim=-1)(attention_scores)

        context_layer = torch.matmul(attention_probs, value_layer)
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)
        attention_output = self.out_proj(context_layer)

        return attention_output

class MllamaVisionMLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.fc1 = nn.Linear(config.vision_hidden_size, config.vision_intermediate_size)
        self.fc2 = nn.Linear(config.vision_intermediate_size, config.vision_hidden_size)
        self.act_fn = nn.GELU()

    def forward(self, hidden_states):
        hidden_states = self.fc1(hidden_states)
        hidden_states = self.act_fn(hidden_states)
        hidden_states = self.fc2(hidden_states)
        return hidden_states

class MllamaForCausalLM(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.transformer = MllamaTextModel(config)
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)

    def forward(self, input_ids, attention_mask=None, vision_input=None):
        transformer_outputs = self.transformer(input_ids, attention_mask, vision_input)
        hidden_states = transformer_outputs[0]
        lm_logits = self.lm_head(hidden_states)
        return lm_logits

class MllamaTextModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.embed_tokens = nn.Embedding(config.vocab_size, config.hidden_size)
        self.layers = nn.ModuleList([MllamaDecoderLayer(config) for _ in range(config.num_hidden_layers)])
        self.norm = MllamaTextRMSNorm(config.hidden_size, eps=config.rms_norm_eps)

    def forward(self, input_ids, attention_mask=None, vision_input=None):
        hidden_states = self.embed_tokens(input_ids)
        
        if vision_input is not None:
            # Concatenate vision input with text embeddings
            hidden_states = torch.cat([vision_input, hidden_states], dim=1)
            
            if attention_mask is not None:
                # Create attention mask for vision input (allowing attending to all vision tokens)
                vision_attention_mask = torch.ones(
                    (vision_input.size(0), 1, 1, vision_input.size(1)),
                    dtype=attention_mask.dtype,
                    device=attention_mask.device
                )
                # Concatenate vision attention mask with text attention mask
                attention_mask = torch.cat([vision_attention_mask, attention_mask], dim=-1)

        for layer in self.layers:
            hidden_states = layer(hidden_states, attention_mask)

        hidden_states = self.norm(hidden_states)
        return (hidden_states,)

class MllamaDecoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.self_attn = MllamaTextSelfSdpaAttention(config)
        self.mlp = MllamaTextMLP(config)
        self.input_layernorm = MllamaTextRMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.post_attention_layernorm = MllamaTextRMSNorm(config.hidden_size, eps=config.rms_norm_eps)

    def forward(self, hidden_states, attention_mask=None):
        residual = hidden_states
        hidden_states = self.input_layernorm(hidden_states)
        hidden_states = self.self_attn(hidden_states, attention_mask=attention_mask)
        hidden_states = residual + hidden_states

        residual = hidden_states
        hidden_states = self.post_attention_layernorm(hidden_states)
        hidden_states = self.mlp(hidden_states)
        hidden_states = residual + hidden_states

        return hidden_states

class MllamaTextSelfSdpaAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.num_heads = config.num_attention_heads
        self.head_dim = config.hidden_size // config.num_attention_heads
        self.q_proj = nn.Linear(config.hidden_size, config.num_attention_heads * self.head_dim, bias=False)
        self.k_proj = nn.Linear(config.hidden_size, config.num_attention_heads * self.head_dim, bias=False)
        self.v_proj = nn.Linear(config.hidden_size, config.num_attention_heads * self.head_dim, bias=False)
        self.o_proj = nn.Linear(config.num_attention_heads * self.head_dim, config.hidden_size, bias=False)

    def forward(self, hidden_states, attention_mask=None):
        batch_size, seq_length, _ = hidden_states.size()

        q = self.q_proj(hidden_states).view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.k_proj(hidden_states).view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.v_proj(hidden_states).view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)

        scores = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(self.head_dim)
        if attention_mask is not None:
            scores = scores + attention_mask

        attn_weights = F.softmax(scores, dim=-1)
        context_layer = torch.matmul(attn_weights, v)

        context_layer = context_layer.transpose(1, 2).contiguous().view(batch_size, seq_length, -1)
        attn_output = self.o_proj(context_layer)

        return attn_output

class MllamaTextMLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.gate_proj = nn.Linear(config.hidden_size, config.intermediate_size, bias=False)
        self.up_proj = nn.Linear(config.hidden_size, config.intermediate_size, bias=False)
        self.down_proj = nn.Linear(config.intermediate_size, config.hidden_size, bias=False)
        self.act_fn = nn.SiLU()

    def forward(self, x):
        return self.down_proj(self.act_fn(self.gate_proj(x)) * self.up_proj(x))

class MllamaTextRMSNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-6):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.epsilon = eps

    def forward(self, hidden_states):
        variance = hidden_states.to(torch.float32).pow(2).mean(-1, keepdim=True)
        hidden_states = hidden_states * torch.rsqrt(variance + self.epsilon)
        return self.weight * hidden_states.to(self.weight.dtype)


@dataclass
class MllamaConfig:
    # Vision model config
    image_size: int = 224
    patch_size: int = 14
    num_channels: int = 3
    vision_hidden_size: int = 1280
    vision_intermediate_size: int = 5120
    num_vision_attention_heads: int = 16
    num_vision_layers: int = 32

    # Language model config
    vocab_size: int = 128264
    hidden_size: int = 4096
    intermediate_size: int = 14336
    num_attention_heads: int = 32
    num_hidden_layers: int = 40
    max_position_embeddings: int = 2048

    # Cross-model config
    num_cross_attention_layers: int = 8

    # Other
    rms_norm_eps: float = 1e-6
    initializer_range: float = 0.02

# Utility functions
def create_attention_mask_from_input_ids(input_ids, padding_idx=1):
    """Create attention mask from input_ids."""
    attention_mask = (input_ids != padding_idx).long()
    attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
    attention_mask = (1.0 - attention_mask) * torch.finfo(torch.float32).min
    return attention_mask

def create_position_ids_from_input_ids(input_ids, padding_idx=1):
    """Create position IDs from input IDs."""
    mask = input_ids.ne(padding_idx).long()
    incremental_indices = torch.cumsum(mask, dim=1).long() * mask
    return incremental_indices.long() + padding_idx

def load_weights_into_modified_model(config):
    # Load the pretrained model
    pretrained_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-11B-Vision")
    pretrained_config = pretrained_model.config

    # Update our config with values from the pretrained config
    for key, value in pretrained_config.to_dict().items():
        if hasattr(config, key):
            setattr(config, key, value)

    # Initialize our modified model
    modified_model = MllamaForConditionalGeneration(config)

    # Helper function to set weights with size check
    def set_weight(new_module, old_module, prefix=''):
        for name, param in old_module.named_parameters():
            full_name = f"{prefix}.{name}" if prefix else name
            if hasattr(new_module, full_name):
                new_param = getattr(new_module, full_name)
                if isinstance(new_param, torch.nn.Parameter) and param.size() == new_param.size():
                    new_param.data.copy_(param.data)
                else:
                    print(f"Size mismatch for {full_name}. Skipping.")
            else:
                print(f"Parameter {full_name} not found in new model. Skipping.")

    # Load weights for vision_model_1 and vision_model_2
    if hasattr(pretrained_model, 'vision_model'):
        set_weight(modified_model.vision_model_1, pretrained_model.vision_model)
        set_weight(modified_model.vision_model_2, pretrained_model.vision_model)
    else:
        print("No vision model found in pretrained model. Initializing vision models randomly.")

    # Load weights for language_model
    set_weight(modified_model.language_model, pretrained_model)

    # Load weights for multi_modal_projector
    if hasattr(pretrained_model, 'multi_modal_projector'):
        old_projector = pretrained_model.multi_modal_projector
        new_projector = modified_model.multi_modal_projector
        if old_projector.weight.size(0) == new_projector.weight.size(0):
            # If output size matches, duplicate the input connections
            new_projector.weight.data = torch.cat([old_projector.weight.data, old_projector.weight.data], dim=1)
            if hasattr(new_projector, 'bias') and new_projector.bias is not None:
                new_projector.bias.data.copy_(old_projector.bias.data)
        else:
            print(f"Size mismatch for multi_modal_projector. Initializing randomly.")
    else:
        print("No multi_modal_projector found in pretrained model. Initializing randomly.")

    # Initialize weights for vision_cross_attention randomly
    # (This is a new component, so we keep the random initialization)

    print("Weights loaded into the modified model.")
    return modified_model



config = MllamaConfig()

# Load weights into our modified model
modified_model = load_weights_into_modified_model(config)

print("Modified model with loaded weights:")
print(modified_model)

# Example input (you would need to replace these with actual data)
batch_size = 1
seq_length = 20
image_1 = torch.randn(batch_size, config.num_channels, config.image_size, config.image_size)
image_2 = torch.randn(batch_size, config.num_channels, config.image_size, config.image_size)
input_ids = torch.randint(0, config.vocab_size, (batch_size, seq_length))
attention_mask = create_attention_mask_from_input_ids(input_ids)

# Forward pass
outputs = model(image_1, image_2, input_ids, attention_mask)

# Process outputs (e.g., get predicted token)
predicted_token = outputs.argmax(dim=-1)

print(f"Shape of model output: {outputs.shape}")
print(f"Predicted token: {predicted_token}")

Loading checkpoint shards: 100%|██████████| 5/5 [00:00<00:00,  7.67it/s]
Some weights of MllamaForCausalLM were not initialized from the model checkpoint at meta-llama/Llama-3.2-11B-Vision and are newly initialized: ['embed_tokens.weight', 'layers.0.input_layernorm.weight', 'layers.0.mlp.down_proj.weight', 'layers.0.mlp.gate_proj.weight', 'layers.0.mlp.up_proj.weight', 'layers.0.post_attention_layernorm.weight', 'layers.0.self_attn.k_proj.weight', 'layers.0.self_attn.o_proj.weight', 'layers.0.self_attn.q_proj.weight', 'layers.0.self_attn.v_proj.weight', 'layers.1.input_layernorm.weight', 'layers.1.mlp.down_proj.weight', 'layers.1.mlp.gate_proj.weight', 'layers.1.mlp.up_proj.weight', 'layers.1.post_attention_layernorm.weight', 'layers.1.self_attn.k_proj.weight', 'layers.1.self_attn.o_proj.weight', 'layers.1.self_attn.q_proj.weight', 'layers.1.self_attn.v_proj.weight', 'layers.10.input_layernorm.weight', 'layers.10.mlp.down_proj.weight', 'layers.10.mlp.gate_proj.weight', 'layers.10.mlp.u

No vision model found in pretrained model. Initializing vision models randomly.
Parameter model.embed_tokens.weight not found in new model. Skipping.
Parameter model.layers.0.self_attn.q_proj.weight not found in new model. Skipping.
Parameter model.layers.0.self_attn.k_proj.weight not found in new model. Skipping.
Parameter model.layers.0.self_attn.v_proj.weight not found in new model. Skipping.
Parameter model.layers.0.self_attn.o_proj.weight not found in new model. Skipping.
Parameter model.layers.0.mlp.gate_proj.weight not found in new model. Skipping.
Parameter model.layers.0.mlp.up_proj.weight not found in new model. Skipping.
Parameter model.layers.0.mlp.down_proj.weight not found in new model. Skipping.
Parameter model.layers.0.input_layernorm.weight not found in new model. Skipping.
Parameter model.layers.0.post_attention_layernorm.weight not found in new model. Skipping.
Parameter model.layers.1.self_attn.q_proj.weight not found in new model. Skipping.
Parameter model.layers.1

NameError: name 'model' is not defined